In [1]:
%load_ext autoreload
%autoreload 2
%aimport -numpy
%aimport -torch
%aimport -gym

import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
torch.manual_seed(0)

# for saving/loading
import os

import base64, io

# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob

In [2]:
from Train_policy_func import PolicyContinuous, device
from Generate_traj_func import generate_trajectory
from Plot_Functions import plot_suboptimality, plot_trajectory_performance, plot_Scores
from PPO import ppo_update, RolloutBuffer, evaluate_policy

# Training loop for OPPO
from OPPO import baseline_CartPole_V0, baseline_MountainCar_continuous, OPPO_update
# from OPPO import baseline_1, baseline_CartPole_v0_Fla

c:\Users\turin\OneDrive\EPFL\Master\MA2\RL\RLHF\Git RLHF\RL-RLHF\Plot_Functions.py:19: DeprecationWarning: invalid escape sequence '\p'
  plt.plot(updates1, reward_hist_pi1, label=f"$\pi_1$ Scores \n mean = {np.mean(reward_hist_pi1):.1f}")


In [3]:
# Choose environment:

# env 3:  MountainCarContinuous,

env_name='MountainCarContinuous'
env_oppo = gym.make(env_name)
baseline = baseline_MountainCar_continuous
state_dim  = env_oppo.observation_space.shape[0]  # e.g. 2
action_dim = env_oppo.action_space.shape[0]       # e.g. 1
policy = PolicyContinuous(state_dim, action_dim).to(device)
target_score = 180 # TODO

# pass bounds into act (optional)
low, high = env_oppo.action_space.low, env_oppo.action_space.high


load_policies = False

# ── Hyperparams for OPPO ──
lr_oppo = 0.003 # learning rate for OPPO
n_episodes              = 1200 # keep same “number of iterations” for fair comparison
max_t                   = 200  # cap on steps per episode
print_every             = 20   # print every x episodes and avg also the score for every x episodes
gamma                   = 0.99

seed_training_policies  = 42


# ── Instantiate the two policies & optimizers for OPPO ──

pi1_oppo = PolicyContinuous(state_size=env_oppo.observation_space.shape[0],
                     action_size=env_oppo.action_space.n).to(device)
opt1_oppo    = torch.optim.Adam(pi1_oppo.parameters(), lr=lr_oppo)

pi2_oppo = PolicyContinuous(state_size=env_oppo.observation_space.shape[0],
                     action_size=env_oppo.action_space.n).to(device)
opt2_oppo   = torch.optim.Adam(pi2_oppo.parameters(), lr=lr_oppo)


# ── Load the policies if you want to reuse them ──
if load_policies:
  pi1_oppo.load_state_dict(torch.load(f"pi1_oppo_{env_name}.pth"))


# ── Run the OPPO training ──
if not load_policies:
  # Training the policy:
  scores_oppo = OPPO_update(
      policy       = pi1_oppo,
      optimizer    = opt1_oppo,
      env          = env_oppo,
      baseline     = baseline,
      n_episodes   = n_episodes,
      max_t        = max_t,
      gamma        = gamma,      # you can reuse your PPO γ
      print_every  = print_every,
      early_stop   = False,
      seed = seed_training_policies,
      target_score = target_score,
      env_name= env_name,
      display_every = False
  )

pi2_oppo.load_state_dict(torch.load(f"pi2_oppo_{env_name}.pth"))


seed_evaluation = 26
num_episodes = 100
pi1_mean_reward, pi1_reward = evaluate_policy(pi1_oppo, env_oppo, n_episodes=num_episodes)
pi2_mean_reward, pi2_reward = evaluate_policy(pi2_oppo, env_oppo, n_episodes=num_episodes)
print("Evaluations over", num_episodes, "episodes done for both policies")

plot_suboptimality(
    pi1_reward,
    pi2_reward,
    max_reward=200, # for CartPole-v0
)

plot_Scores(
    pi1_reward,
    pi2_reward,
)


C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\envs\registration.py:601: UserWarning: WARN: Using the latest versioned environment `MountainCarContinuous-v0` instead of the unversioned environment `MountainCarContinuous`.
  logger.warn(
C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\turin\AppData\Roaming\Python\Python311\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: 'Box' object has no attribute 'n'